<a href="https://colab.research.google.com/github/saifmdev/stock_scraper_ta/blob/main/stock_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Simple OHLC Chart with Pandas and Plotly

##Run this command if 'ta' library now installed

In [ ]:
pip install ta

In [ ]:
pip install plotly 

##Import Libraries

In [ ]:
from ta.trend import ema_indicator as ema
from ta.volume import volume_weighted_average_price as vwap
from ta.momentum import rsi
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
from datetime import date, datetime
import time


In [ ]:
#Sample list of tickers
tickers = ['AA','MSFT', 'FB', 'TSLA', 'GOOG', 'AMZN']

#create from and to timestamps in epoch
current  = date.today().strftime('%s')
past = int(datetime(2019,12,1,0,0).timestamp())

#loop through tickers and query yahoo finance website to get .csv 
for ticker in tickers:
  link = f"https://query1.finance.yahoo.com/v7/finance/download/{ticker}?period1={past}&period2={current}&interval=1d&events=history&includeAdjustedClose=true"
  df = pd.read_csv(link)
  df['EMA13'] = ema(df['Close'],13)
  df['EMA21'] = ema(df['Close'],21)
  df['EMA34'] = ema(df['Close'],34)
  df['EMA55'] = ema(df['Close'],55)
  df['EMA89'] = ema(df['Close'],89)
  df['EMA144'] = ema(df['Close'],144)
  df['VWAP'] = vwap(df['High'], df['Low'],df['Close'], df['Volume'],5)
  fig = px.line(df, x="Date", y="EMA13", title=ticker, labels={ticker})
  fig.add_scatter(x=df['Date'], y=df['EMA21'], mode='lines', name="EMA21")
  fig.add_scatter(x=df['Date'], y=df['EMA34'], mode='lines', name="EMA34")
  fig.add_scatter(x=df['Date'], y=df['EMA55'], mode='lines', name="EMA55")
  fig.add_scatter(x=df['Date'], y=df['EMA89'], mode='lines', name="EMA89")
  fig.add_scatter(x=df['Date'], y=df['EMA144'], mode='lines', name="EMA144")
  fig.add_scatter(x=df['Date'], y=df['VWAP'], mode='lines', name="VWAP")
  fig.add_ohlc(open=df['Open'], high=df['High'], low=df['Low'], close=df['Close'], x=df["Date"],name="OHLC")
  fig.update_layout(
    yaxis_title='Price in $USD'
  )

  fig.show()

#YAHOO! Finance Web Scraper + INDICATOR CALCULATIONS

##Run this command if 'ta' library not installed

In [3]:
pip install ta

  Created wheel for ta: filename=ta-0.9.0-py3-none-any.whl size=28908 sha256=52acaf6c1b6156f82e884de6a87c32fe6ec6f3c8ec6a60266e2aee90b8be2c03
  Stored in directory: /root/.cache/pip/wheels/72/78/64/cc1c01506a1010a9845e9bd7c69333730f7174661228ea4f98
Successfully built ta


##Run this command if 'plotly' library not installed

In [2]:
pip install plotly

##Import Libraries

In [16]:
from datetime import date, datetime
from ta.trend import ema_indicator as ema, sma_indicator as sma, wma_indicator as wma, macd,macd_diff,macd_signal
from ta.volume import volume_weighted_average_price as vwap, on_balance_volume as obv
from ta.momentum import rsi
from ta.volatility import average_true_range as atr
from urllib.error import HTTPError
from pathlib import Path
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import numpy as np
from scipy.stats import linregress
import os
import shutil
import time

In [17]:
#Hull Moving Average Calculation using Weighted Moving Average (Fibonacci Band - 13,34,55,89,144)

def hma(s, period):
  return wma(wma(s, period//2).multiply(2).sub(wma(s, period)), int(np.sqrt(period)))

In [20]:
#Set from and to date in epoch format (int)
current  = date.today().strftime('%s')
past = int(datetime(2019,1,1,0,0).timestamp())

#Get tickers from csv and convert to numpy array
ticker_list = 'https://drive.google.com/uc?export=download&id=1xzFbp9nmKQL0vz2F_BkVzdZercgLfEsq'
tickers_df = pd.read_csv(ticker_list)
tickers = tickers_df['Symbol'].to_numpy()

#Set OPENCLOSEDIFF_AVG Period
open_close_diff_period = 5

#remove existing directory and create new empty one for update
try:
    p = Path('/tickers')
    if p.exists():
      shutil.rmtree('./tickers')
    time.sleep(0.5)
    os.makedirs('tickers', exist_ok=True)  
except OSError as e:
    print("Error: %s - %s." % (e.filename, e.strerror))

#Loop through array of tickers to make queries to Yahoo Finance and save results as .csv
for ticker in tickers:
  try:
    link = f"https://query1.finance.yahoo.com/v7/finance/download/{ticker}?period1={past}&period2={current}&interval=1d&events=history&includeAdjustedClose=true"
    # time.sleep(1)
    data = pd.read_csv(link)
    #Calculate Difference between Close and Open and Moving Average for smoothing (Period: 5)
    data['OPENCLOSEDIFF'] = data['Close'] - data['Open']
    data['OPENCLOSEDIFF_SMA_5'] = sma(data['OPENCLOSEDIFF'],open_close_diff_period)
    data['OPENCLOSEDIFF_SMA_AVG_5'] = sma(data['OPENCLOSEDIFF_SMA_5'],open_close_diff_period)
    #Calculate Exponential Moving Average Fibonacci Band and Calculate Average of all values combined - TREND IND.
    data['EMA13'] = ema(data['Close'],13)
    data['EMA21'] = ema(data['Close'],21)
    data['EMA34'] = ema(data['Close'],34)
    data['EMA55'] = ema(data['Close'],55)
    data['EMA89'] = ema(data['Close'],89)
    data['EMA144'] = ema(data['Close'],144)
    data['EMA_AVG'] = (data['EMA13'] + data['EMA34'] + data['EMA55'] + data['EMA89'] + data['EMA144'])/5
    #Calculate Hull Moving Average Fibonacci Band and Calculate Average of all values combined - TREND IND.
    data['HMA13'] = hma(data['Close'],13)
    data['HMA34'] = hma(data['Close'],34)
    data['HMA55'] = hma(data['Close'],55)
    data['HMA89'] = hma(data['Close'],89)
    data['HMA144'] = hma(data['Close'],144)
    data['HMA_AVG'] = (data['HMA13'] + data['HMA34'] + data['HMA55'] + data['HMA89'] + data['HMA144'])/5
    #Calculate MACD, MACD Signal Line and MACD Histogram - TREND IND.
    data['MACD'] = macd(data['Close'],26,12)
    data['MACD_Signal'] = macd_signal(data['Close'],26,12)
    data['MACD_Histogram'] = macd_diff(data['Close'],26,12)
    #Calculate OBV
    data['OBV'] = obv(data['Close'], data['Volume'])
    data['OBV_EMA_13'] = ema(data['OBV'],13)
    data['OBV_EMA_34'] = ema(data['OBV'],34)
    data['OBV_EMA_55'] = ema(data['OBV'],55)
    #Calculate VWAP - VOLUME IND.
    data['VWAP'] = vwap(data['High'], data['Low'],data['Close'], data['Volume'],5)
    #Calculate RSI Fibonacci Band and Average of values combined - MOMENTUM IND.
    data['RSI8'] = rsi(data['Close'],8)
    data['RSI13'] = rsi(data['Close'],13)
    data['RSI21'] = rsi(data['Close'],21)
    data['RSI_AVG'] = (data['RSI13'] + data['RSI13'] + data['RSI21']) / 3
    #Calculate Average True Range
    data['ATR'] = atr(data['High'], data['Low'], data['Close'], 14)
    #Write File to tikcers directory
    data.to_csv(f"./tickers/{ticker}.csv")
    print(f"{ticker} Downloaded ✓")
  except HTTPError as err:
    print(f"{ticker} Not Found X")
print("Download Complete")

A Downloaded ✓
AA Downloaded ✓
AAC Downloaded ✓
AACG Downloaded ✓
AACI Downloaded ✓
AACIW Not Found X


KeyboardInterrupt: ignored

#Test Block

In [7]:
import plotly.express as px
import pandas as pd

In [9]:
df = pd.read_csv('./tickers/AA.csv')

fig1 = px.line(df, x="Date", y="HMA_AVG", title="HMA_AVG")
fig1.add_ohlc(open=df['Open'], high=df['High'], low=df['Low'], close=df['Close'], x=df["Date"],name="OHLC")

fig1.show()

fig2 = px.line(df, x="Date", y="OPENCLOSEDIFF_SMA_AVG_5", title="OPENCLOSEDIFF MOVING AVERAGE - 5 DAYS PERIOD")
fig2.add_hline(y=0, line_width=3, line_dash="dash", line_color="green")

fig2.show()